In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl
import nltk
import pickle

nltk.download('punkt')
nltk.download('wordnet')

!git clone https://github.com/wookylee/ML_for_spam_text_selection.git
from google.colab import drive
drive.mount('/content/drive/')

filename = '/content/drive/My Drive/기계학습/train.csv'
data = pd.read_csv(filename)


import nltk as nlp
import re
description_list = []
for description in data['message']:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)


from sklearn.feature_extraction.text import CountVectorizer 
max_features = 1000 #We use the most common word
count_vectorizer = CountVectorizer( max_features = max_features ,stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))
features = count_vectorizer.get_feature_names()
print(count_vectorizer.get_feature_names())

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import cross_val_predict, cross_val_score

def print_score(classifier,X_train,y_train,X_test,y_test,train=True):
    if train == True:
        print("Training results:\n")
        print('Accuracy Score: {0:.4f}\n'.format(accuracy_score(y_train,classifier.predict(X_train))))
        print(sum(classifier.predict(X_train)))
        #print('Classification Report:\n{}\n'.format(classification_report(y_train,classifier.predict(X_train))))
        #print('Confusion Matrix:\n{}\n'.format(confusion_matrix(y_train,classifier.predict(X_train))))
        res = cross_val_score(classifier, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy')
        print(res)
        print('Average Accuracy:\t{0:.4f}\n'.format(res.mean()))
        print('Standard Deviation:\t{0:.4f}'.format(res.std()))
    elif train == False:
        print("Test results:\n")
        print('Accuracy Score: {0:.4f}\n'.format(accuracy_score(y_test,classifier.predict(X_test))))
        #print('Classification Report:\n{}\n'.format(classification_report(y_test,classifier.predict(X_test))))
        #print('Confusion Matrix:\n{}\n'.format(confusion_matrix(y_test,classifier.predict(X_test))))


#We separate our data is train and test
y = data.iloc[:,1].values   # male or female classes

x = sparce_matrix
print(sparce_matrix)
print(len(sparce_matrix))
print(len(sparce_matrix[0]))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)
y

#We make model for predict
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)
print("the accuracy of our model: {}".format(nb.score(X_test,y_test)))

from sklearn.neighbors import KNeighborsClassifier as KNN

knn = KNN()
knn.fit(X_train,y_train)
print('----------KNeighborClassifier----------')
print("our accuracy is: {}".format(knn.score(X_test,y_test)))
print_score(knn,X_train,y_train,X_test,y_test,train=True)
print_score(knn,X_train,y_train,X_test,y_test,train=False)

from sklearn.svm import SVC
svc = SVC(kernel='rbf',random_state=42)

svc.fit(X_train,y_train)
print('----------SupportVectorClassifier----------')
print("our accuracy is: {}".format(svc.score(X_test,y_test)))
print_score(svc,X_train,y_train,X_test,y_test,train=True)
print_score(svc,X_train,y_train,X_test,y_test,train=False)

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
rfc.fit(X_train, y_train)
print('----------RandomForestClassifier----------')
print("our accuracy is: {}".format(rfc.score(X_test,y_test)))
print_score(rfc,X_train,y_train,X_test,y_test,train=True)
print_score(rfc,X_train,y_train,X_test,y_test,train=False)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(X_train,y_train)
#lr.fit(x,y)
print('----------LogisticRegression----------')
print("our accuracy is: {}".format(lr.score(X_test,y_test)))
print_score(lr,X_train,y_train,X_test,y_test,train=True)
print_score(lr,X_train,y_train,X_test,y_test,train=False)

from sklearn.tree import DecisionTreeClassifier as DT

dtc = DT(criterion='entropy',random_state=42)
dtc.fit(X_train,y_train)
print('----------DecisionTreeClassifier----------' )
print("our accuracy is: {}".format(dtc.score(X_test,y_test)))
print_score(dtc,X_train,y_train,X_test,y_test,train=True)
print_score(dtc,X_train,y_train,X_test,y_test,train=False)

from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image

list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
print(models.head(n=10))

best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]
models[models['Test Precision']==1].head(n=5)

best_index = models[models['Test Precision']==1]['Test Accuracy'].idxmax()
print('###############################')
print(best_index)
print(list_alpha[best_index])
print('###############################')
bayes = naive_bayes.MultinomialNB(alpha=list_alpha[best_index])
bayes.fit(X_train, y_train)
models.iloc[best_index, :]


# filename = '/content/drive/My Drive/기계학습/RFC.pkl'
# pickle.dump(rfc, open(filename, 'wb'))

# filename = '/content/drive/My Drive/기계학습/Bayes.pkl'
# pickle.dump(bayes, open(filename, 'wb'))

# filename = '/content/drive/My Drive/기계학습/LR.pkl'
# pickle.dump(lr, open(filename, 'wb'))

# filename = '/content/drive/My Drive/기계학습/features.pkl'
# pickle.dump(count_vectorizer.get_feature_names(), open(filename, 'wb'))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Cloning into 'ML_for_spam_text_selection'...
Mounted at /content/drive/
the most using 1000 words: ['abiola', 'able', 'abt', 'ac', 'access', 'account', 'actually', 'add', 'address', 'admirer', 'aft', 'afternoon', 'age', 'ago', 'ah', 'ahead', 'aight', 'air', 'al', 'alex', 'alright', 'alrite', 'amp', 'angry', 'answer', 'anymore', 'anytime', 'apply', 'appreciate', 'april', 'ar', 'ard', 'area', 'arrive', 'asap', 'ask', 'asked', 'asking', 'attempt', 'auction', 'available', 'ave', 'await', 'award', 'awarded', 'away', 'awesome', 'babe', 'baby', 'bad', 'bag', 'bak', 'balance', 'bank', 'bath', 'bathe', 'battery', 'bb', 'bcoz', 'beautiful', 'bed', 'bedroom', 'begin', 'believe', 'belly', 'best', 'better', 'bid', 'big', 'bird', 'birthday', 'bit', 'black', 'blood', 'blue', 'bluetooth', 'bo

alpha             5.940010
Train Accuracy    0.985278
Test Accuracy     0.987500
Test Recall       0.909091
Test Precision    1.000000
Name: 54, dtype: float64

In [26]:
# filename2 = '/content/drive/My Drive/기계학습/leaderboard_test_file.csv'
# eval = pd.read_csv(filename2)

# description_list2 = []
# for description in eval['message']:
#     description = re.sub("[^a-zA-Z]"," ",description)
#     description = description.lower()   
#     description = nlp.word_tokenize(description)
#     #description = [ word for word in description if not word in set(stopwords.words("english"))]
#     lemma = nlp.WordNetLemmatizer()
#     description = [ lemma.lemmatize(word) for word in description]
#     description = " ".join(description)
#     description_list2.append(description)


# from sklearn.feature_extraction.text import CountVectorizer 
# max_features = 6000 #We use the most common word
# count_vectorizer = CountVectorizer( vocabulary = features ,stop_words = "english")
# sparce_matrix = count_vectorizer.fit_transform(description_list2).toarray()
# print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))
# print(count_vectorizer.get_feature_names())
# print(sparce_matrix)
# print(len(sparce_matrix))
# print(len(sparce_matrix[0]))

# # if len(sparce_matrix[0])<5995:
# #     margin_matrix = np.zeros((len(sparce_matrix),5995-len(sparce_matrix[0])))
# #     sparce_matrix = np.hstack((sparce_matrix,margin_matrix))
# # print(sparce_matrix)
# # print(len(sparce_matrix))
# # print(len(sparce_matrix[0]))

# x_pred = sparce_matrix.reshape(300,5995,1)

# result = regressor.predict_classes(x_pred)

# print(result)